サンプリング

In [9]:
import pyro.distributions as dist
import torch
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO

In [10]:
def model(data):
    # define the hyperparameters that control the beta prior
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the bernoulli
        # likelihood Bernoulli(f)
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

In [4]:
def model(data):
    # ベータ事前分布からfをサンプリング
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data using pyro.sample with the obs keyword argument
    for i in range(len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

 To explicitly mark this in Pyro we basically just need to replace the Python builtin range with the Pyro construct plate:

In [5]:
def model(data):
    # sample f from the beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data [WE ONLY CHANGE THE NEXT LINE]
    for i in pyro.plate("data_loop", len(data)):
        # observe datapoint i using the bernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

In [6]:
with plate('observe_data'):
    pyro.sample('obs', dist.Bernoulli(f), obs=data)

NameError: name 'plate' is not defined

グローバル潜在変数、ローカル潜在変数(グラフィカルモデル